In [ ]:
from app.data.reference import (filename_stakedao_delegations, 
                                filename_stakedao_locker, 
                                filename_stakedao_staked_sdcrv, 
                                known_large_cvx_holders_addresses
                                )
from datetime import datetime as dt

from app.data.local_storage import (
    pd,
    read_json,
    read_csv,
    write_dataframe_csv,
    write_dfs_to_xlsx,
    csv_to_df,
    write_dataframe_csv
    )
from app.utilities.utility import (
    get_period_direct, 
    get_period_end_date, 
    get_date_obj, 
    get_dt_from_timestamp,
    shift_time_days,
    df_remove_nan
)

In [ ]:
import plotly
import plotly.express as px

import plotly.graph_objects as go
from plotly.subplots import make_subplots


In [ ]:
from app.stakedao.snapshot.models import df_stakedao_snapshot_vote_choice

In [ ]:
def get_df(filename):
    df = csv_to_df(filename, 'raw_data')

    # df_gauge_pool_map = df_gauge_pool_map.sort_values("BLOCK_TIMESTAMP", axis = 0, ascending = True)
    return df

In [ ]:
df_delegations = get_df(filename_stakedao_delegations)

In [ ]:
df_delegations

In [ ]:
df_locker = get_df(filename_stakedao_locker)

In [ ]:
df_locker

Types:
* Deposit
* Supply
* Withdraw

### Supply 
* block_timestamp,
* contract_name,
* event_name,
    * previous_supply,
    * value,
* origin_from_address,
* tx_hash

### Deposit
* block_timestamp,
* contract_name,
* event_name,
    * provider,
    * value,
    * ts,
    * locktime,
    * type,
* origin_from_address,
* tx_hash

### Withdraw
* block_timestamp,
* contract_name,
* event_name,
    * provider,
    * value,
    * ts,
* origin_from_address,
* tx_hash

# Process StakeDAO veSDT

In [ ]:
class ProcessStakeDAOveSDT():
    def __init__(self, df):
        self.df = df
        self.process_types()
        pass

    def process_types(self):
        try:
            self.df['block_timestamp'] = self.df['block_timestamp'].apply(get_date_obj)
            self.df['value'] = self.df['value'].atype(float)
            self.df['ts'] = self.df['ts'].apply(get_dt_from_timestamp)
            self.df['known_as'] = self.df['user'].apply(lambda x: self.known_as(x))
        except:
            self.df['BLOCK_TIMESTAMP'] = self.df['BLOCK_TIMESTAMP'].apply(get_date_obj)
            self.df['VALUE'] = self.df['VALUE'].atype(float)
            self.df['TS'] = self.df['TS'].apply(get_dt_from_timestamp)
            self.df['KNOWN_AS'] = self.df['USER'].apply(lambda x: self.known_as(x))

    def process_df(self):
        for i, row in self.df.iterrows():
            if 'deposit':
                pass


    def known_as(self, user):
        if user in known_large_cvx_holders_addresses:
            return known_large_cvx_holders_addresses[user]
        else:
            return "_"
    
class Deposit():
    def __init__(self, row):
        self.row = row
        self.block_timestamp = row['block_timestamp'] if 'block_timestamp' in row else row['BLOCK_TIMESTAMP']
        self.contract_name = row['contract_name'] if 'contract_name' in row else row['CONTRACT_NAME']
        self.provider = row['provider'] if 'provider' in row else row['PROVIDER']
        self.value = row['value'] if 'value' in row else row['VALUE']
        self.ts = row['ts'] if 'ts' in row else row['TS']
        self.origin_from_address = row['origin_from_address'] if 'origin_from_address' in row else row['ORIGIN_FROM']
        self.tx_hash = row['tx_hash'] if 'tx_hash' in row else row['TX_HASH']
        self.known_as = row['known_as'] if 'known_as' in row else row['KNOWN_AS']


# class Supply():
#     def __init__(self):
#         pass


class Withdraw():
    def __init__(self, row):
        self.row = row
        self.block_timestamp = row['block_timestamp'] if 'block_timestamp' in row else row['BLOCK_TIMESTAMP']
        self.contract_name = row['contract_name'] if 'contract_name' in row else row['CONTRACT_NAME']
        self.provider = row['provider'] if 'provider' in row else row['PROVIDER']
        self.value = row['value'] if 'value' in row else row['VALUE']
        self.origin_from_address = row['origin_from_address'] if 'origin_from_address' in row else row['ORIGIN_FROM']
        self.tx_hash = row['tx_hash'] if 'tx_hash' in row else row['TX_HASH']
        self.known_as = row['known_as'] if 'known_as' in row else row['KNOWN_AS']







# Staked sdCRV

In [ ]:
df_staked_sdcrv = get_df(filename_stakedao_staked_sdcrv)

In [ ]:
df_staked_sdcrv

In [ ]:
class StakeDAOsdCRV():
    def __init__(self, df):
        self.df = self.sort(df)
        self.processed_df = []
        self.processed_agg = []
        self.processed_agg_known = []
        self.process_df()
        self.process_agg()
        self.process_agg_known_as()

    def sort(self, df=[], target='block_timestamp'):
        if len(df) == 0:
            return df
        return df.sort_values([target], axis = 0, ascending = True)

    def adjust_withdraws(self, row):
        if row['event_name'] == 'Deposit':
            return row['value']
        elif row['event_name'] == 'Withdraw':
            return -1 * row['value']
        
    def known_as(self, user):
        if user in known_large_cvx_holders_addresses:
            return known_large_cvx_holders_addresses[user]
        else:
            return "_"
        
    def process_df(self):
        processed_df = self.df.copy()

        processed_df['value'] = processed_df['value'].astype(float)
        processed_df['known_as'] = processed_df['provider'].apply(lambda x: self.known_as(x))

        processed_df['balance_delta'] = processed_df.apply(lambda x: self.adjust_withdraws(x), axis=1)
        processed_df['staked_balance'] = processed_df.groupby(
            ['provider']
            )['balance_delta'].transform(pd.Series.cumsum)
        
        processed_df['date'] = processed_df['block_timestamp'].apply(get_date_obj).dt.date


        temp = processed_df.sort_values('date').groupby(['known_as']).tail(1)
        temp['date'] = temp.date.max()
        temp['balance_delta'] = 0

        processed_df = pd.concat([processed_df, temp])
        processed_df = self.sort(processed_df, 'block_timestamp')

        self.processed_df = processed_df

    def process_agg(self):
        processed_agg = self.processed_df[['date', 'balance_delta']].groupby([
                'date'
            ]).agg(
            balance_delta=pd.NamedAgg(column='balance_delta', aggfunc=sum),
        ).reset_index()
        processed_agg = self.sort(processed_agg, 'date')
        processed_agg['total_staked_balance'] = processed_agg['balance_delta'].transform(pd.Series.cumsum)
        self.processed_agg = processed_agg


    def process_agg_known_as(self):
        processed_agg_known = self.processed_df[['known_as', 'date', 'balance_delta']].groupby([
                'date', 'known_as'
            ]).agg(
            balance_delta=pd.NamedAgg(column='balance_delta', aggfunc=sum),
        ).reset_index()



        processed_agg_known = self.sort(processed_agg_known, 'date')
        processed_agg_known['staked_balance'] = processed_agg_known.groupby('known_as')['balance_delta'].transform(pd.Series.cumsum)
        # set end value of current balance w/o delta from last delta

        self.processed_agg_known = processed_agg_known


In [ ]:
sdcrv = StakeDAOsdCRV(df_staked_sdcrv)

In [ ]:
fig = px.area(sdcrv.processed_agg,
                x=sdcrv.processed_agg['date'],
                y=sdcrv.processed_agg['total_staked_balance'],
                # color='known_as',
                # line_shape='linear',
                # facet_row=facet_row,
                # facet_col_wrap=facet_col_wrap
                )
# fig.add_vline(x=dt.now(), line_width=2, line_dash="dash", line_color="black")
fig.update_layout(
    title=f"StakeDAO sdCRV Total Staked",
        xaxis_title="Date",
        yaxis_title="Staked sdCRV",
    #     legend_title="Legend Title",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="RebeccaPurple"
    ),
    # height= 1000,
)
fig.show()

In [ ]:
fig = px.line(sdcrv.processed_agg,
                x=sdcrv.processed_agg['date'],
                y=sdcrv.processed_agg['balance_delta'],
                # color='known_as',
                # line_shape='linear',
                # facet_row=facet_row,
                # facet_col_wrap=facet_col_wrap
                )
# fig.add_vline(x=dt.now(), line_width=2, line_dash="dash", line_color="black")
fig.update_layout(
    title=f"Staked sdCRV Total Balance Changes",
        xaxis_title="Date",
        yaxis_title="Staked sdCRV Balance Delta",
    #     legend_title="Legend Title",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="RebeccaPurple"
    ),
    # height= 1000,
)
fig.show()

In [ ]:
fig = px.line(sdcrv.processed_agg_known,
                x=sdcrv.processed_agg_known['date'],
                y=sdcrv.processed_agg_known['staked_balance'],
                color='known_as',
                # line_shape='linear',
                # facet_row=facet_row,
                # facet_col_wrap=facet_col_wrap
                )
fig.add_vline(x=dt.now(), line_width=2, line_dash="dash", line_color="black")
fig.update_layout(
    title=f"StakeDAO Staked sdCRV Balances",
        xaxis_title="This Date",
        yaxis_title="Staked sdCRV Balance",
    #     legend_title="Legend Title",
    font=dict(        family="Courier New, monospace",
        size=18,
        color="RebeccaPurple"
    ),
    height= 1000,
)
fig.show()